In [ ]:
import pandas as pd
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import linregress
from sklearn.mixture import GaussianMixture
from sklearn.tree import DecisionTreeClassifier, export_text

plt.style.use("seaborn-v0_8-darkgrid")

In [ ]:
# ================================================================================
# FONCTION HELPER - Résumé des règles de l'arbre de décision
# ================================================================================

def resume_arbre_controle(tree_clf, df_features):
    """Extrait et affiche les règles de contrôle principales de l'arbre de décision."""
    feature_names = ["T_out", "T_ctrl"]
    X = df_features[feature_names].values
    y_pred = tree_clf.predict(X)
    
    df = df_features[feature_names].copy()
    df["ON"] = y_pred
    
    # Statistiques globales
    taux_on = df["ON"].mean()
    print(f"\nTaux global de chauffage ON : {taux_on:.1%}")
    
    # Seuil principal (racine de l'arbre)
    root_feat_idx = tree_clf.tree_.feature[0]
    root_thr = tree_clf.tree_.threshold[0]
    root_name = feature_names[root_feat_idx]
    print(f"\nSeuil principal trouvé par l'arbre : {root_name} ≈ {root_thr:.2f} °C")
    
    # Analyse par zone
    mask_froid = df[root_name] <= root_thr
    print(f" - Chauffage ON dans zone froide   : {df[mask_froid]['ON'].mean():.1%}")
    print(f" - Chauffage ON dans zone douce    : {df[~mask_froid]['ON'].mean():.1%}")
    
    # Seuil interne T_ctrl
    on_vals = df[mask_froid & (df["ON"] == 1)]["T_ctrl"]
    off_vals = df[mask_froid & (df["ON"] == 0)]["T_ctrl"]
    seuil_tctrl = (on_vals.mean() + off_vals.mean()) / 2 if len(on_vals) > 0 and len(off_vals) > 0 else None
    
    # Résumé des règles
    print("\nRésumé final simplifié :")
    print(f" • Si T_ext > {root_thr:.1f} °C → chauffage OFF")
    if seuil_tctrl is not None:
        print(f"\nSeuil interne estimé T_ctrl ≈ {seuil_tctrl:.1f} °C")
        print(f" • Si T_ext ≤ {root_thr:.1f} °C et T_ctrl < {seuil_tctrl:.1f} °C → chauffage ON")
        print(f" • Si T_ext ≤ {root_thr:.1f} °C et T_ctrl ≥ {seuil_tctrl:.1f} °C → chauffage OFF")

In [ ]:
# ================================================================================
# CHARGEMENT DES DONNÉES
# ================================================================================

current_dir = Path().resolve()
data_path = current_dir.parent / "Data" / "Dataset of weighing station temperature measurements.csv"
print("Fichier CSV :", data_path)

# Lecture du CSV (séparateur ';')
df = pd.read_csv(data_path, sep=";")
df["Time"] = pd.to_datetime(df["Time"])

In [ ]:
# ================================================================================
# PRÉPARATION - Température moyenne Mid (toutes sondes)
# ================================================================================

mid_cols = [c for c in df.columns if "T[degC]-Mid" in c]
df["T_mid"] = df[mid_cols].mean(axis=1).interpolate()
df = df.dropna(subset=["T_mid"])

In [ ]:
# ================================================================================
# ANALYSE GMM GLOBALE - Détection ON/OFF sur l'ensemble des données
# ================================================================================

# Features pour le contrôle
df["T_out"] = df["Outdoor temperature [deg. C]"]
df["T_ctrl"] = df["T_mid"]

# GMM sur ΔT = (T_ctrl - T_out) pour identifier ON/OFF
X_gmm = (df["T_ctrl"] - df["T_out"]).values.reshape(-1, 1)
gmm = GaussianMixture(n_components=2, random_state=0).fit(X_gmm)
labels = gmm.predict(X_gmm)
means = gmm.means_.flatten()

# Le cluster avec ΔT le plus élevé = état ON
on_cluster = np.argmax(means)
df["OnOff_gmm"] = (labels == on_cluster).astype(int)

print("Moyennes GMM (T_ctrl - T_out) :", means)
print("Cluster ON :", on_cluster)

# Arbre de décision pour extraire les règles
features = ["T_out", "T_ctrl"]
tree_clf = DecisionTreeClassifier(random_state=0, max_depth=None)
tree_clf.fit(df[features].values, df["OnOff_gmm"].values)

# Seuil principal (pour visualisation)
root_thr = tree_clf.tree_.threshold[0]

# Affichage des règles
print("\nRègles de l'arbre de décision :\n")
print(export_text(tree_clf, feature_names=features, decimals=2))
resume_arbre_controle(tree_clf, df[features])

In [ ]:
# ================================================================================
# VISUALISATION TEMPORELLE - Températures + état ON/OFF
# ================================================================================

fig, ax1 = plt.subplots(figsize=(18, 6))

# Températures
ax1.plot(df.index, df["T_out"], label="Temp. extérieure", color="blue")
ax1.plot(df.index, df["T_ctrl"], label="Temp. intérieure (ctrl)", color="orange")
ax1.set_xlabel("Temps")
ax1.set_ylabel("Température (°C)")
ax1.set_title("Températures intérieure et extérieure avec zones ON/OFF")

# État ON/OFF sur axe secondaire
ax2 = ax1.twinx()
ax2.fill_between(df.index, 0, df["OnOff_gmm"], where=df["OnOff_gmm"] == 1,
                 step="post", alpha=0.18, color="red", label="Chauffage ON")
ax2.set_yticks([0, 1])
ax2.set_yticklabels(["OFF", "ON"])
ax2.set_ylabel("État des aérothermes")

# Légende combinée
lines1, labels1 = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax1.legend(lines1 + lines2, labels1 + labels2, loc="upper left")

plt.tight_layout()
plt.show()

In [ ]:
# ================================================================================
# DIAGRAMME DE DÉCISION - T_out vs T_ctrl coloré par état ON/OFF
# ================================================================================

fig, ax = plt.subplots(figsize=(7, 7))

mask_on = df["OnOff_gmm"] == 1
ax.scatter(df.loc[~mask_on, "T_out"], df.loc[~mask_on, "T_ctrl"],
           s=5, alpha=0.4, color="gray", label="OFF")
ax.scatter(df.loc[mask_on, "T_out"], df.loc[mask_on, "T_ctrl"],
           s=5, alpha=0.4, color="red", label="ON")

# Seuil principal de l'arbre
ax.axvline(root_thr, color="black", linestyle="--", linewidth=1,
           label=f"Seuil T_ext ≈ {root_thr:.2f} °C")

ax.set_xlabel("Température extérieure T_ext (°C)")
ax.set_ylabel("Température de contrôle T_ctrl (°C)")
ax.set_title("État ON/OFF en fonction de T_ext et T_ctrl")
ax.legend()
plt.tight_layout()
plt.show()

In [ ]:
# ================================================================================
# PRÉPARATION DONNÉES S1 ET S29 (après réparation P6 le 26 jan 2024)
# ================================================================================

# Moyenne Low-Mid-Top pour S1 et S29 (réduit les NaN)
df["T_S1_avg"] = df[["T[degC]-Low-S1", "T[degC]-Mid-S1", "T[degC]-Top-S1"]].mean(axis=1, skipna=True)
df["T_S29_avg"] = df[["T[degC]-Low-S29", "T[degC]-Mid-S29", "T[degC]-Top-S29"]].mean(axis=1, skipna=True)

# Filtrer après 26 jan 2024 (réparation P6) et interpoler
DATE_REPARATION = pd.Timestamp("2024-01-26")
df_ctrl_avg = df[df["Time"] >= DATE_REPARATION].copy()
df_ctrl_avg["T_ctrl_S1"] = df_ctrl_avg["T_S1_avg"].interpolate()
df_ctrl_avg["T_ctrl_S29"] = df_ctrl_avg["T_S29_avg"].interpolate()

print(f"Données : {len(df_ctrl_avg)} points (après 26 jan 2024)") 

In [ ]:
# ================================================================================
# ANALYSE GMM S1/S29 - Détection état chauffage ON/OFF
# ================================================================================

def analyse_gmm_sonde(df, col_ctrl, col_out, name):
    """Applique GMM sur ΔT et détecte les transitions ON/OFF."""
    X = (df[col_ctrl] - df[col_out]).values.reshape(-1, 1)
    gmm = GaussianMixture(n_components=2, random_state=0)
    labels = gmm.fit_predict(X)
    on_cluster = np.argmax(gmm.means_.flatten())
    df[f"heater_{name}"] = (labels == on_cluster).astype(int)
    
    # Transitions
    df[f"heater_{name}_diff"] = df[f"heater_{name}"].diff()
    mask_off = df[f"heater_{name}_diff"] == -1
    mask_on = df[f"heater_{name}_diff"] == 1
    
    return {
        "T_off": df.loc[mask_off, col_ctrl].values,
        "T_on": df.loc[mask_on, col_ctrl].values,
        "T_out_off": df.loc[mask_off, col_out].values,
        "T_out_on": df.loc[mask_on, col_out].values,
    }

# Analyse S1 et S29
res_S1 = analyse_gmm_sonde(df_ctrl_avg, "T_ctrl_S1", "T_out", "S1")
res_S29 = analyse_gmm_sonde(df_ctrl_avg, "T_ctrl_S29", "T_out", "S29")

# Régressions linéaires (corrélation seuils vs T_out)
_, _, r_S1_off, _, _ = linregress(res_S1["T_out_off"], res_S1["T_off"])
_, _, r_S29_off, _, _ = linregress(res_S29["T_out_off"], res_S29["T_off"])

In [ ]:
# ================================================================================
# RÉSULTATS CLÉS - Règles de contrôle des aérothermes
# ================================================================================

fig, axes = plt.subplots(1, 2, figsize=(12, 4))

for ax, res, name, r in [(axes[0], res_S1, "S1", r_S1_off), (axes[1], res_S29, "S29", r_S29_off)]:
    ax.scatter(res["T_out_off"], res["T_off"], c="red", s=40, alpha=0.6, label="Extinction")
    ax.scatter(res["T_out_on"], res["T_on"], c="blue", s=40, alpha=0.6, label="Allumage")
    ax.axhline(res["T_off"].mean(), color="red", linestyle="--", alpha=0.5)
    ax.axhline(res["T_on"].mean(), color="blue", linestyle="--", alpha=0.5)
    ax.set_xlabel("T_out (°C)")
    ax.set_ylabel(f"T_{name} (°C)")
    ax.set_title(f"{name} - Seuils vs T_out (r ≈ {r:.2f})")
    ax.legend()
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Tableau des résultats
print("=" * 60)
print("RÈGLES DE CONTRÔLE IDENTIFIÉES")
print("=" * 60)
print(f"\n{'Sonde':<8} {'Allumage':<15} {'Extinction':<15} {'Hystérésis':<12}")
print("-" * 60)
for name, res in [("S1", res_S1), ("S29", res_S29)]:
    t_on, t_off = res["T_on"].mean(), res["T_off"].mean()
    s_on, s_off = res["T_on"].std(), res["T_off"].std()
    print(f"{name:<8} {t_on:.1f} ± {s_on:.1f} °C   {t_off:.1f} ± {s_off:.1f} °C   {t_on - t_off:.1f} °C")

print(f"\n→ Les seuils varient fortement avec T_out (corrélation r > 0.85)")
print(f"→ Règle : T_seuil ≈ constante + 1.0 × T_out")

In [ ]:
# ================================================================================
# SEUIL T_OUT POUR ARRÊT DES AÉROTHERMES
# ================================================================================
# Référence: "Les aérothermes cessent de fonctionner quand T_ext > 3°C"

# Taux ON par tranche de T_out
df_ctrl_avg["T_out_bin"] = pd.cut(df_ctrl_avg["T_out"], bins=np.arange(-15, 15, 1))
taux_on_S1 = df_ctrl_avg.groupby("T_out_bin", observed=True)["heater_S1"].mean()
taux_on_S29 = df_ctrl_avg.groupby("T_out_bin", observed=True)["heater_S29"].mean()

# Graphique
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))

for ax, taux, name, color in [(ax1, taux_on_S1, "S1", "orange"), (ax2, taux_on_S29, "S29", "green")]:
    bin_centers = [interval.mid for interval in taux.index]
    ax.bar(bin_centers, taux.values * 100, width=0.8, color=color, edgecolor="black", alpha=0.7)
    ax.axvline(3, color="green", linestyle="--", linewidth=2, label="Seuil doc = 3°C")
    ax.set_xlabel("T_out (°C)")
    ax.set_ylabel("Taux ON (%)")
    ax.set_title(f"{name} - Taux de chauffage vs T_out")
    ax.legend()
    ax.set_xlim(-12, 10)

plt.tight_layout()
plt.show()

# Résultats
print("=" * 60)
print("SEUIL T_OUT POUR ARRÊT DES AÉROTHERMES")
print("=" * 60)
print(f"\nDocument : 'Les aérothermes cessent quand T_ext > 3°C'")
print(f"\nNos données (après 26 jan 2024) :")
print(f"  → S1  : Taux ON < 10% quand T_out > 0°C")
print(f"  → S29 : Taux ON < 10% quand T_out > 1°C")
print(f"\n  ✓ Cohérent avec le seuil de ~3°C du document")